### Importing libraries

In [3]:
import open3d, os
import ctypes
from ctypes import *
from glob import glob

# Dataset

Since our dataset is quite big and it take several hours to preprocess it, we have uploaded the preprocessed dataset in a zip file. You can download it from [here](https://drive.google.com).
Be sure to extract the zip file and upload the dataset to the working directory.

# Preprocessing
Skip this section if you have downloaded the preprocessed dataset and go directly to the training of neural network.

#### Data augmentation

First of all we need to augment the data in order to prevent overfitting.
First step is to rotate and scale (without changing the aspect ratio) the meshes.

In [4]:
from utils import rotate_scale

files = glob("./new_dataset/train/input/quads/*")
for file in files:
    rotate_scale(file)


Secondly, we generate the trimesh objects from the quads meshes. This is done because the input of our network must be a trimesh object.

In [5]:
from utils import generate_trimesh

files = glob("./new_dataset/train/input/quads/*")

for file in files:
    generate_trimesh(file)


Lastyl, we generate the output (true lables) of the network, which are the direction fields of the vertices.

In [6]:
from utils import generate_output
import numpy as np

files_quads = glob("./new_dataset/train/input/quads/*")

for file in files_quads:
  file_name = os.path.basename(file)
  file_triangle = "./new_dataset/train/input/triangles/" + file_name

  orientation_fields = generate_output(file_triangle, file)
  orientation_fields_reshaped = orientation_fields.reshape(orientation_fields.shape[0], -1)


  # save the orientation fields in a txt file
  output_file = "./new_dataset/train/output/" + file_name
  np.savetxt(output_file, orientation_fields_reshaped)


  print("Orientation fields saved in: ", output_file)

## Loading mesh to retopologize

In [7]:
# loading the shared library with C++
shared_library_path = glob(os.path.join("./build", "**/*.so"), recursive=True)
assert shared_library_path, "No shared library found, please build the shared library first (README.md for instructions)"
shared_library = ctypes.CDLL(shared_library_path[0])

file_path = "./new_dataset/train/input/triangles"
file_output_path = os.path.join(file_path, "..", "..", "output")

files_triangles = glob("./new_dataset/train/input/triangles/*")

for file_triangle in files_triangles:

    mesh = open3d.io.read_triangle_mesh(file_triangle)

    vertices = mesh.vertices
    triangles = mesh.triangles

    num_vertices = len(vertices)
    num_triangles = len(triangles)

    # create file input.txt
    file = open("input.txt", "w")

    # write vertices
    for i in range(len(vertices)):
        file.write(str(vertices[i][0]) + " " + str(vertices[i][1]) + " " + str(vertices[i][2]) + "\n")

    # write triangles
    for i in range(len(triangles)):
        file.write(str(triangles[i][0]) + " " + str(triangles[i][1]) + " " + str(triangles[i][2]) + "\n")


    file.close()

    k = 3 # neighborhood size

    file_output = os.path.join(file_output_path, os.path.basename(file_triangle))

    # get the complete url of the file
    file_output = os.path.abspath(file_output)

    # modify the basename concatenating _principal_directions
    file_output = file_output.replace(".obj", "_principal_directions.txt")

    output = create_string_buffer(file_output.encode('utf-8'),size=100)
    shared_library.crest_lines(num_vertices, num_triangles, k, output)

AssertionError: No shared library found, please build the shared library first (README.md for instructions)

# Learning

General settings

In [ ]:
import torch
import os
from pathlib import Path

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using CUDA...")
else:
    device = torch.device('cpu')
    print("Using CPU...")

dtype = torch.float32

# Problem/dataset things
n_class = 4

# Model 
input_features = 'hks'      # 'hks' or 'xyz'
k_eig = 128

# Training settings
train = True                # Whether to train the model
n_epoch = 10000
lr = 1e-4
decay_every = 50
decay_rate = 0.5
augment_random_rotate = (input_features == 'xyz')

# Important paths
base_path = Path(os.getcwd())
op_cache_dir = os.path.join(base_path, "neural_network", "data", "op_cache")
dataset_path = os.path.join(base_path, "new_dataset")
dataset_path = os.path.normpath(dataset_path)

Loading data

In [ ]:
from torch.utils.data import DataLoader
from neural_network.dataset import MeshDataset

# Load the test dataset
test_dataset = MeshDataset(dataset_path, train=False, k_eig=k_eig, use_cache=False, op_cache_dir=op_cache_dir)
test_loader = DataLoader(test_dataset, batch_size=None)

# Load the train dataset
if train:
    train_dataset = MeshDataset(dataset_path, train=True, k_eig=k_eig, use_cache=False, op_cache_dir=op_cache_dir)
    train_loader = DataLoader(train_dataset, batch_size=None, shuffle=True)

Model settings

In [ ]:
from neural_network import diffusion_net

# === Create the model

C_in={'xyz':3, 'hks':12}[input_features] # Dimension of input features

model = diffusion_net.layers.DiffusionNet(C_in=C_in,
                                          C_out=n_class,
                                          C_width=128, 
                                          N_block=5,
                                          outputs_at='vertices', 
                                          dropout=True)

if os.path.exists('saved_model.pth'):
    model.load_state_dict(torch.load('saved_model.pth', map_location=device))
    print('Loaded model from: saved_model.pth')

model = model.to(device)

# === Optimize

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

Training

In [ ]:
from neural_network.neural_network_management import train_epoch, test

if train:
    print("Training...")

    for epoch in range(n_epoch):
        if epoch % 100 == 0 and epoch > 0:
            torch.save(model.state_dict(), 'saved_model.pth')
            print('Model saved in: saved_model.pth')
        train_loss = train_epoch(epoch, decay_every, decay_rate, optimizer, model, train_loader, device, augment_random_rotate, input_features)
        test_loss = test(model, test_loader, device, input_features)
        print("Epoch {} - Train overall: {:06.3f}  Test overall: {:06.3f}".format(epoch, train_loss, test_loss))


test_loss = test()
print("Overall test loss: {:06.3f}%".format(test_loss))